In [128]:
## Importing The Dependencies ##
import pickle as pk
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier




## Creating Our Data Frames ##
df = pd.read_csv('train.csv')
df_val = pd.read_csv('test.csv')


## Get Name Function For Both Data Frames ##
def get_title(name):
    try:
        if '.' in name:
            return name.split(',')[1].split('.')[0].strip()
        else:
            return "Unknown"
    except:
        print("Something Went Wrong")

## A list with the all the different titles For Both ##
titles = sorted(set([x for x in df.Name.map(lambda x: get_title(x))]))
titles_val = sorted(set([x for x in df_val.Name.map(lambda x: get_title(x))]))


## Normalize the titles ##
def replace_titles(x):
    try:
        title = x['Title']
        if title in ['Capt', 'Col', 'Major']:
            return "Officer"
        elif title in ["Jonkheer","Don",'the Countess', 'Dona', 'Lady',"Sir"]:
            return "Royalty"
        elif title in ['the Countess', 'Mme', 'Lady']:
            return "Mrs"
        elif title in ['Mlle', 'Ms']:
            return "Miss"
        else:
            return title
    except:
        print("Oh Fudge Functions Crashed!")


try:
    ## Lets create a new column for the titles For Both ##
    df['Title'] = df['Name'].map(lambda x: get_title(x))
    df_val['Title'] = df_val['Name'].map(lambda x: get_title(x))


    ## And replace the titles For Both, so the are normalized to 'Mr', 'Miss' and 'Mrs' ##
    df['Title'] = df.apply(replace_titles, axis=1)
    df_val['Title'] = df_val.apply(replace_titles, axis=1)
    ids = df_val['PassengerId']


    ## Doing data Science Stuff To Clean The Data For Both ##
    ## Main (train.csv) Data Frame:
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna("S", inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df.drop("Cabin", axis=1, inplace=True)
    df.drop("Name", axis=1, inplace=True)
    df.drop("Ticket", axis=1, inplace=True)
    df.Sex.replace(('male', 'female'), (0,1), inplace=True)
    df.Embarked.replace(('S', 'C', 'Q'), (0,1,2), inplace=True)
    df.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'), (0,1,2,3,4,5,6,7), inplace=True)

    ## Secondary (test.csv) DataFrame:
    df_val['Age'].fillna(df_val['Age'].median(), inplace=True)
    df_val['Fare'].fillna(df_val['Fare'].median(), inplace=True)
    df_val['Embarked'].fillna("S", inplace=True)
    df_val.drop("Cabin", axis=1, inplace=True)
    df_val.drop("Name", axis=1, inplace=True)
    df_val.drop("Ticket", axis=1, inplace=True)
    df_val.drop("PassengerId", axis=1, inplace=True)
    df_val.Sex.replace(('male', 'female'), (0,1), inplace=True)
    df_val.Embarked.replace(('S', 'C', 'Q'), (0,1,2), inplace=True)
    df_val.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'), (0,1,2,3,4,5,6,7), inplace=True)


    ## Prediction and model creating ##
    predictors = df.drop(['PassengerId', 'Survived'], axis=1)
    target = df['Survived']
    x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size=0.22, random_state=0)

    randomforest = RandomForestClassifier()
    randomforest.fit(x_train, y_train)
    y_pred = randomforest.predict(x_val)
    acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
    print("Accuracy: {}".format(acc_randomforest))


    ## Not Sure For This Part ##
    pk.dump(randomforest, open("SOP_Model.sav", 'wb'))



    ## Secndary (test.csv) DataFrame:
    predictions = randomforest.predict(df_val)
    output = pd.DataFrame({ 'PssengerId' : ids, 'Survived' : predictions})
    output.to_csv('submission.csv', index=False)
except:
    print("SomeThing Is Not Working Out :(")

Accuracy: 82.23
